# Scikit-time objects

This notebook showcases two PySINDy objects designed for use in [Scikit-time](https://scikit-time.github.io/index.htm):
* `SINDyEstimator` - TODO
* `SINDyModel` - TODO

In [1]:
import numpy as np
from scipy.integrate import odeint

import pysindy as ps

In [2]:
def lorenz(z, t):
    return [
        10 * (z[1] - z[0]),
        z[0] * (28 - z[2]) - z[1],
        z[0] * z[1] - (8 / 3) * z[2]
    ]

# Generate measurement data
dt = .002

# Train data
t_train = np.arange(0, 10, dt)
x0_train = [-8, 8, 27]
x_train = odeint(lorenz, x0_train, t_train)

# Test data

# Evolve the Lorenz equations in time using a different initial condition
t_test = np.arange(0, 15, dt)
x0_test = np.array([8, 7, 15])
x_test = odeint(lorenz, x0_test, t_test)

In [3]:
# Fit an estimator
estimator = ps.scikit_time.SINDyEstimator(t_default=dt)
estimator.fit(x_train);

In [4]:
# Extract a model
model = estimator.fetch_model()

# Compare SINDy-predicted derivatives with finite difference derivatives
print('Model score: %f' % model.score(x_test, t=dt))

# Evolve the new initial condition in time with the SINDy model
x_test_sim = model.simulate(x0_test, t_test)

Model score: 1.000000


In [6]:
# Try out some other combinations of methods
estimator = ps.scikit_time.SINDyEstimator(
    optimizer=ps.SR3(threshold=0.5, max_iter=50),
    feature_library=ps.PolynomialLibrary(degree=3)
)
estimator.fit(x_train, t=dt)

model = estimator.fetch_model()